In [2]:
import requests
import json
import pandas as pd
from json_excel_converter import Converter 
from json_excel_converter.xlsx import Writer
from datetime import datetime
import os
import warnings

warnings.filterwarnings('ignore')

categoryList = {}
categoryList['1'] = "TP01" #CODICE
categoryList['2'] = "TP02" #FIBRA
categoryList['3'] = "TP03" #CESSIONE CREDITI
categoryList['4'] = "TP04" #NCB
categoryList['5'] = "TP05" #DELEGA
categoryList['6'] = "TP06" #FONDI
categoryList['7'] = "TP07" #
categoryList['8'] = "TP08" #ECOBONUS
categoryList['9'] = "TP09" #CARTA DI DEBITO



#### INPUT ###########
inCustomer = input("Inserisci CF/PIVA (invio per skip):\n")
if inCustomer == '' : 
    inDossier = input("Inserisci DossierId (invio per skip):\n")
flag=0

if inCustomer != '' : 
    print(f'Ricerca per Customer: {inCustomer}')
elif inDossier != '' :   
    print(f'Ricerca per Dossier: {inDossier}')
else:
    print('Inserire almeno un campo di ricerca')
    flag = 1


if flag == 0 :     
    if inCustomer != '' : 
        inCategory = str(input("Inserisci Categoria (invio per skip):\n 1:CODICE\n 2:FIBRA\n 3:CESSIONE CREDITI\n 4:NCB\n 5:DELEGA\n 6:FONDI\n 7:FINANZIAMENTI\n 8:ECO BONUS\n 9:CARTA DO DEBITO\n"))
    
    urlBaseAPI = "https://sviluppo-apim-common.azure-api.net/internal/coll/crmu-dossier/" 
   
    # setting API GET retrieveDossiers
    url = urlBaseAPI + "api/v1/dossier/retrieve"
    if inCustomer != '' and inCategory != '': 
        payload = "{'customerReference':{'classification':{'code':'R'},'taxCode':'"+inCustomer+"'}, 'category': '"+categoryList[inCategory]+"'}"
    elif inCustomer != '' and inCategory == '': 
        payload = "{'customerReference':{'classification':{'code':'R'},'taxCode':'"+inCustomer+"'}}"
    else:
        payload = "{'dossierId':'"+inDossier+"'}"
    
    headers = {'cache-control': "no-cache",'Postman-Token': "82274d00-2048-4f0a-9239-ac2ca0c060d4",'Host': "sviluppo-apim-common.azure-api.net",'Content-Type': "application/json"}

    response = requests.request("POST", url, verify=False, data=payload, headers=headers)
    dataResp = response.json()
    if dataResp['info']['resultCode'] != '404':
        dossiers = dataResp['dossiers']
        print("Numero Dossier Trovati: "+ str(len(dossiers))+"\n")
    else:
        print("Nessun Dossier Trovato")
        flag = 1
        
    if flag == 0:    
        #creating path
        now = datetime.now()
        date_time = now.strftime("%d-%m-%Y")
        
        path = "/Collaudo/" + date_time
        path1 = "/Collaudo/" + date_time + "/dossierFull"
        path2 = "/Collaudo/" + date_time + "/assetFull"
        basePath = os.getcwd()


        try:
            os.makedirs(basePath + path1)
        except OSError:
            print ("Creation of the directory %s failed" % path1)
        else:
            print ("Successfully created the directory %s " % path1)
        
        try:
            os.makedirs(basePath + path2)
        except OSError:
            print ("Creation of the directory %s failed" % path2)
        else:
            print ("Successfully created the directory %s " % path2)
        


        checkDossier = []
        assetsId = {}


        for dossier in dossiers: 
            dossierId = dossier['customData']['dossierId']
            nomeFile= basePath + path1 + "/dossierFull_" + dossierId 

            #creazione dossierFull
            with open(nomeFile + ".json", 'w', encoding='utf-8') as f:
                json.dump(dossier, f, ensure_ascii=False, indent=4) 
                print(nomeFile + ".json Creato correttamente" )  

            #checkDossier
            dictDossier = {}
            dictDossier['dossierId'] = dossierId
            dictDossier['dossierNumber'] = dossier['id']
            dictDossier['transferMode'] = dossier['transferMode']
            dictDossier['createdOn'] = dossier['createdOn']
            dictDossier['checkDossier'] = {}

            dictCheck = {}

            #checkDossier element
            if 'category' in dossier :
                dictDossier['category'] = dossier['category']
                dictCheck['checkCategory'] = "OK"
            else:
                dictCheck['checkCategory'] = "KO"

            if 'status' in dossier :
                dictDossier['status'] = dossier['status']
                dictCheck['checkStatus'] = "OK"
            else:
                dictCheck['checkStatus'] = "KO"

            if 'source' in dossier :
                dictDossier['source'] = dossier['source']
                dictCheck['checkSource'] = "OK"
            else:
                dictCheck['checkSource'] = "KO"

            if 'operation' in dossier :
                dictDossier['operation'] = dossier['operation']
                dictCheck['checkOperation'] = "OK"
            else:
                dictCheck['checkOperation'] = "KO"

            if 'channel' in dossier :
                dictDossier['channel'] = dossier['channel']
                dictCheck['checkChannel'] = "OK"
            else:
                dictCheck['checkChannel'] = "KO"

            #checkCliente
            if 'customData' in dossier :
                customData = dossier['customData']
                if 'customerReference' in customData :
                    customerReference = customData['customerReference']
                    if 'bicCode' in customerReference :
                        dictDossier['bicCode'] = customerReference['bicCode']
                        dictCheck['checkBicCode'] = "OK"
                    else: 
                        dictCheck['checkBicCode'] = "KO"

                    if 'taxCode' in customerReference :
                        dictDossier['taxCode'] = customerReference['taxCode']
                        dictCheck['checkTaxCode'] = "OK"
                    else: 
                        dictCheck['checkTaxCode'] = "KO"

            #checkDocument
            if 'dossierItems' in dossier:
                dossierItem = dossier['dossierItems'][0]
                if 'documents' in dossierItem:
                    if len(dossierItem['documents']) > 0:
                        dictCheck['checkDocuments'] = "OK"
                    else:
                        dictCheck['checkDocuments'] = "KO"

                #checkProduct
                if 'productCode' in dossierItem:
                    dictDossier['productCode'] = dossierItem['productCode']
                    dictCheck['productCode'] = "OK"
                else: 
                    dictCheck['productCode'] = "KO"

                if 'productName' in dossierItem:
                    dictDossier['productName'] = dossierItem['productName']
                    dictCheck['productName'] = "OK"
                else: 
                    dictCheck['productName'] = "KO"


                #checkAsset
                if 'asset' in dossierItem:
                    asset = dossierItem['asset']
                    dictDossier['assetId'] = asset['id']
                    assetsId[asset['id']] = asset['id']


            dictDossier['checkDossier'] = dictCheck   
            checkDossier.append(dictDossier)



        #Converting in xls
        conv = Converter()
        nowX = datetime.now()
        timestampX = str(int(datetime.timestamp(nowX)))
        conv.convert(checkDossier, Writer(file= basePath + path + "/dossierHeader_"+timestampX+".xlsx"))
        print("File dossierHeader_"+timestampX+".xlsx creato correttamente\n")
        


        #preparing for assetFull 
        print("Numero Asset associati Trovati: "+ str(len(assetsId))+"\n")
        urlBaseAPI2 = "https://sviluppo-apim-common.azure-api.net/internal/coll/crmu-contract/" 
        url = urlBaseAPI2 + "api/v1/asset/retrieve"

        headers = {
            'cache-control': "no-cache",
            'Postman-Token': "82274d00-2048-4f0a-9239-ac2ca0c060d4",
            'Host': "sviluppo-apim-common.azure-api.net",
            'Content-Type': "application/json"
            }

        for assetId in assetsId:
            payload = "{'assetId':'"+assetId+"'}"
            response = requests.request("POST", url, verify=False, data=payload, headers=headers)
            dataResp = response.json()
            if 'assets' in dataResp:
                nomeFile= basePath + path2 + "/assetFull_" + assetId 
                with open(nomeFile + ".json", 'w', encoding='utf-8') as f:
                    json.dump(dataResp, f, ensure_ascii=False, indent=4) 
                    print(nomeFile + ".json Creato correttamente" )  

            else:
                print("no result for id: "+ assetId)

else:
    print("FINE")



Inserisci CF/PIVA (invio per skip):
PMPNDR79C13A944Y
Ricerca per Customer: PMPNDR79C13A944Y
Inserisci Categoria (invio per skip):
 1:CODICE
 2:FIBRA
 3:CESSIONE CREDITI
 4:NCB
 5:DELEGA
 6:FONDI
 7:FINANZIAMENTI
 8:ECO BONUS
 9:CARTA DO DEBITO

Numero Dossier Trovati: 16

Successfully created the directory /Collaudo/14-10-2021/dossierFull 
Successfully created the directory /Collaudo/14-10-2021/assetFull 
C:\Users\GUZZONES/Collaudo/14-10-2021/dossierFull/dossierFull_dcdef31a-42bd-eb11-bacc-000d3ade33e4.json Creato correttamente
C:\Users\GUZZONES/Collaudo/14-10-2021/dossierFull/dossierFull_5049ffa8-59bd-eb11-bacc-000d3ade33e4.json Creato correttamente
C:\Users\GUZZONES/Collaudo/14-10-2021/dossierFull/dossierFull_52a96c4f-34be-eb11-bacc-000d3ade33e4.json Creato correttamente
C:\Users\GUZZONES/Collaudo/14-10-2021/dossierFull/dossierFull_1569f043-35be-eb11-bacc-000d3ade33e4.json Creato correttamente
C:\Users\GUZZONES/Collaudo/14-10-2021/dossierFull/dossierFull_494141f7-d5be-eb11-bacc-000d3